In [10]:
import pandas as pd
import numpy as np

In [11]:
if "snakemake" in locals():
    input_municipalities_path = snakemake.input["municipalities"]
    input_sectors_path = snakemake.input["sectors"]

    input_passenger_survey_path = snakemake.input["survey"]
    input_passenger_daily_path = snakemake.input["daily"]
    input_group_sizes_path = snakemake.input["group_sizes"]

    input_locations_path = snakemake.input["locations"]

    output_municipalities_path = snakemake.output["municipalities"]
    output_sectors_path = snakemake.output["sectors"]
    output_passengers_path = snakemake.output["passengers"]
    output_missing_locations_path = snakemake.output["missing_locations"]

else:
    input_municipalities_path = "../../../results/brussels/census/municipalities.parquet"
    input_sectors_path = "../../../results/brussels/census/sectors.parquet"

    input_passenger_survey_path = "../../../results/brussels/airport/survey.parquet"
    input_passenger_daily_path = "../../../results/brussels/airport/daily_totals.parquet"
    input_group_sizes_path = "../../../results/brussels/airport/group_sizes.parquet"

    input_locations_path = "../../../results/brussels/osm/locations.parquet"

    output_municipalities_path = "../../../results/brussels/marginals/municipalities.parquet"
    output_sectors_path = "../../../results/brussels/marginals/sectors.parquet"
    output_passengers_path = "../../../results/brussels/marginals/passengers.parquet"
    output_missing_locations_path = "../../../results/brussels/marginals/missing_locations.parquet"

In [12]:
# Read input
df_municipalities = pd.read_parquet(input_municipalities_path)
df_sectors = pd.read_parquet(input_sectors_path)

df_daily = pd.read_parquet(input_passenger_daily_path)
df_survey = pd.read_parquet(input_passenger_survey_path)

df_group_sizes = pd.read_parquet(input_group_sizes_path)

df_locations = pd.read_parquet(input_locations_path)

In [13]:
# Reduce municipalities
df_municipalities = df_municipalities.groupby([
    "municipality_id", "age_class", "sex"
])["weight"].sum().reset_index()

# Reduce sectors
df_sectors = df_sectors[["sector_index", "weight"]]

In [14]:
# Transform passenger data
mean_group_size = np.sum(df_group_sizes["group_size"] * df_group_sizes["weight"])
daily_total = df_daily["count"].sum() / mean_group_size

df_survey["weight"] = df_survey["probability"] * daily_total
df_survey["is_passenger"] = True

# Select columns for marginal
df_survey = df_survey[["municipality_id", "sex", "age_class", "is_passenger", "weight"]].copy()

In [15]:
# Make sure that we don't have passengers in the first age category
df_young = df_survey[["municipality_id", "sex"]].drop_duplicates()
df_young["age_class"] = 0
df_young["weight"] = 0.0
df_young["is_passenger"] = True
df_survey = pd.concat([df_survey, df_young])

In [16]:
# Make sure that we don't have passengers in sectors where we don't have locations
df_missing = pd.DataFrame({
    "sector_index": sorted(list(set(df_sectors["sector_index"]) - set(df_locations["sector_index"])))
})

df_missing["is_passenger"] = True
df_missing["weight"] = 0.0

In [17]:
# Output
df_municipalities.to_parquet(output_municipalities_path)
df_sectors.to_parquet(output_sectors_path)
df_survey.to_parquet(output_passengers_path)
df_missing.to_parquet(output_missing_locations_path)